### SQL for extracting items in order of appearance

In [ ]:
SELECT 
      ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank,
      ce.ITEMID,
      di.LABEL AS item_name,
      ce.VALUEUOM,
      COUNT(*) AS entry_count
  FROM read_csv_auto('norm_pneumonia.csv', header=true) AS ce
  JOIN read_csv_auto('D_ITEMS.csv', header=true) AS di
    ON ce.ITEMID = di.ITEMID
  GROUP BY ce.ITEMID, di.LABEL, ce.VALUEUOM
  ORDER BY entry_count DESC;
